In [30]:
import argparse
import numpy as np
import yaml
from multiprocessing import Pool
import sys
from pyGSI.diags import Ozone
from pyGSI.plot_diags import plot_map, plot_histogram
from datetime import datetime

start_time = datetime.now()


def plotting(ozone_config):

    diagfile = ozone_config['ozone input']['path'][0]
    diag_type = ozone_config['ozone input']['data type'][0].lower()
    analysis_use = ozone_config['ozone input']['analysis use'][0]
    layer = ozone_config['ozone input']['layer'][0]
    plot_type = ozone_config['ozone input']['plot type']
    outdir = ozone_config['outdir']

    diag = Ozone(diagfile)

    df = diag.get_data(analysis_use=analysis_use)
    metadata = diag.metadata
    metadata['Diag Type'] = diag_type
    
    column = f'{diag_type}' if diag_type in ['observation'] \
        else f'{diag_type}_adjusted'

    if layer == 0:
        dict_key = 'column total'
    else:
        dict_key = list(df)[layer]
    metadata['Layer'] = dict_key
    
    if analysis_use:
        lats = {
            'assimilated': df[dict_key]['assimilated']['latitude'].to_numpy(),
            'monitored': df[dict_key]['monitored']['latitude'].to_numpy()
        }
        lons = {
            'assimilated': df[dict_key]['assimilated']['longitude'].to_numpy(),
            'monitored': df[dict_key]['monitored']['longitude'].to_numpy()
        }
        
        data = {
            'assimilated': df[dict_key]['assimilated'][column].to_numpy(),
            'monitored': df[dict_key]['monitored'][column].to_numpy()
        }
        
    else:
        lats = df[dict_key]['latitude'].to_numpy()
        lons = df[dict_key]['longitude'].to_numpy()
    
        data = df[dict_key][column].to_numpy()

    if np.isin('histogram', plot_type):
        plot_histogram(data, metadata, outdir)
    if np.isin('spatial', plot_type):
        plot_map(lats, lons, data, metadata, outdir)

In [31]:
config = {
    'ozone input': {
        'analysis use': [True],
        'data type': ['omf'],
        'layer': [0],
        'qc flag': None,
        'path': ['/scratch2/NCEPDEV/stmp1/Kevin.Dougherty/ncDiags/gdas.20200922/00/atmos/diag_omi_aura_ges.2020100200.nc4'],
        'plot type': ['histogram', 'spatial']
    },
    'outdir': './'
}

plotting(config)

/scratch1/NCEPDEV/da/Kevin.Dougherty/anaconda3/envs/UFO/lib/python3.7/site-packages/shapefile.py:385: UserWarning: Shapefile shape has invalid polygon: no exterior rings found (must have clockwise orientation); interpreting holes as exteriors.
  warnings.warn('Shapefile shape has invalid polygon: no exterior rings found (must have clockwise orientation); interpreting holes as exteriors.')
